# MLonMCU

## Prerequisites

Before running this notebook, make sure to run the following steps:

1. Install the required software for your distribution:

   See main `README.md` file!
   
2. Clone this repository

   ```
   git clone TODO
   cd mlonmcu  # enter cloned repository
   ```
   
3. Setup and enter a new virtualenv (Python 3.7+) for using `mlonmcu`:

   ```
   Bla!
   source .venv/bin/activate
   ```
   
   Hint: You can also use a `conda` environment for this.
   
4. Install mlonmcu package

   ```
   make install  # alternative: python setup.py install
   ```
   
5. Install the required python packages:

   ``` 
   # pip install -r requirements.txt  # Done automatically via setup.py?
   pip install -r ipynb/requirements.txt
   ```
   
6. Launch this notebook in jupyter:

   ```
   jupyter notebook ipynb/Demo.ipynb
   ```

## Demonstration (Command Line)

### Preface

This guide should give an idea on the required steps to setup and use MLonMCU using the command line interface. Usage examples are provided as well.

### Initialize a MLonMCU environment

As MLonMCU requires a directory where dependencies, configurations, intermediate artifacts and results are stored, an environment needs to be initialized first. This also allows to use different configuration (e.g. enabled backends or supported features) for multiple environments.

Create and enter an environment diretcory first to initialize it afterwards:

In [4]:
# WARNING: execute this only once! Alternatively remove the environment directory beforehand to start from scratch.
!test -d /tmp/mlonmcu_env && echo "Skipped initialization on environment" \
  || mlonmcu init /tmp/mlonmcu_env --non-interactive

Initializing ML on MCU environment
Selected target directory: /tmp/mlonmcu_env
The directory does not exist! - Created directory.
Creating environment.yml based on template 'default'.
Skipping creation of virtual environment. (already inside one)
Initializing directories in environment: deps
Finished. Please add `export MLONMCU_HOME=/tmp/mlonmcu_env` to your shell configuration to use it anywhere


When not using a notebook, the option `--non-interactive` can be omitted which allows to customize the initialization process interactively.

If the environment directory is not passed to the `mlonmcu` script, it will try to setup a `default` environment in the users home directory (On Ubuntu: `~/.config/mlonmcu/environments/default/`) instead. By providing a `--name` of the virtual environment if will be automatically registered in `~/.config/mlonmcy/environments.ini` for the current user which helps to find and distinguish multiple installed environments on a machine.

In the following the actual environment which should be used by the `mlonmcu` program has to be defined by the user either by defining the environment variable `MLONMCU_HOME` or by using the `-H` (`--home` or `--hint`) command line option. If none of this was done, the following rules are following to look for a suitable environment:

1. Check if `$MLONMCU_HOME` points to a valid environment
2. Use environment provided via the `-H` flag (path or registered name) if available
3. If the local working directory is the root of an environment, use this directory
4. If there is a default environment for the current user, fall back to this one instead

In [5]:
# Use this for bash: `export MLONMCU_HOME=/tmp/mlonmcu_env`
%env MLONMCU_HOME=/tmp/mlonmcu_env

env: MLONMCU_HOME=/tmp/mlonmcu_env


Registered environment can be listed via the command `mlonmcu env list`. In addition there are possibilities to manually update entires in the environments file.

### Customizing the created MLonMCU environment (optional)

The environment was initialized by a template which can be found in the `templates/` directory of the repository chosen via the `--template` flag. Lets inspect the generated configuration file first:

In [6]:
!cat $MLONMCU_HOME/environment.yml

---
# The MLONMCU_HOME is filled in automatically when creating the environment
home: "/tmp/mlonmcu_env"
logging:
  level: DEBUG
  to_file: False
# Default locations for certain directoriescan be changed here
# Non-absolute paths will always be threated relative to the MLONMCU_HOME
paths:
  # Where the dependencies are downloaded and installed
  deps: deps
  # If logging to file is used keep logs in this directory
  logs: logs
  # Location where reports and artifacts are written to
  results: out
  # Directory for intermediate build products, should be located on a large enough drive
  temp: temp
  # A collection of models which will be used to look for models
  # The paths will be checked in the order defined here stopping at the first match
  # Non-existant paths will be skipped without throwing an error
  models:
    - "/tmp/mlonmcu_env/models"
    - "/home/ga87puy/.config/mlonmcu/models"
# Here default clone_urls
repos:
  tensorflow:
    url: "https://github.com/tum-ei-eda/tensorfl

The environment should work out of the box, but feel free to update the contents of the `environment.yml` file to achieve one of the following:

- Change the default paths used for dependencies, results, models,...
- Use different forks/versions of a repository
- Disable certain frameworks/backends to reduce the installation time in the next step.
- Define which features are supported for the given repositories
- Set default backends and targets
- Define a set of default configuration variables

To learn more about this, check out this document: `docs/ENVIRONMENTS.md` (TODO)

### Setting up all MLonMCU dependencies

Depending on the enabled backends and features, a different set of dependencies has to be installed. To reduce the management effort for the user, all these versions are installed and managed automatically using MLonMCU. To trigger this process, run the following in the command line. (This will take a very long time with all frameworks/targets/features enabled when invoked for the first time)

In [8]:
!mlonmcu setup

[2022-01-17 15:34:08]::DEBUG - Enter MlonMcuContext
[2022-01-17 15:34:08]::DEBUG - Locking context
[2022-01-17 15:34:08]::INFO - No cache found in deps directory
[2022-01-17 15:34:09]::DEBUG - Determined dependency order: clone_tensorflow -> clone_tflite_micro_compiler -> clone_etiss -> clone_tvm -> install_llvm -> clone_utvm_staticrt_codegen -> clone_muriscvnn -> install_riscv_gcc -> build_tensorflow -> build_etiss -> install_etiss -> build_tvm -> build_utvm_staticrt_codegen -> build_muriscvnn -> build_etissvp -> build_tflite_micro_compiler
[2022-01-17 15:34:09]::INFO - Processing task: clone_tensorflow
[2022-01-17 15:34:09]::INFO - Processing task: clone_tflite_micro_compiler
[2022-01-17 15:34:09]::INFO - Processing task: clone_etiss
[2022-01-17 15:34:09]::INFO - Processing task: clone_tvm
[2022-01-17 15:34:09]::INFO - Processing task: install_llvm
[2022-01-17 15:34:10]::INFO - Processing task: clone_utvm_staticrt_codegen
[2022-01-17 15:34:10]::INFO - Processing task: clone_muriscvnn

Some things need to be considered here:
- Repositories are only cloned once, so upstream changes or updated urls/branches in the `environments.yml` are not detected automatically
- The flag `--rebuild` can be passed to `mlonmcu setup` to ensure that the setup routine of every dependency is triggered again. However this does not guarentee that all artifacts are properly updated
- To guarantee that the latest versions of a dependency are installed you can remove the `deps` directory manually or alternative use the provided `mlonmcu cleanup` utility. (Execute `mlonmcu cleanup --help` to learn more) 

### Adding models to MLonMCU

By default, the `mlonmcu-models` repository (TODO: url) is cloned to each environment as its contents are designed to perfectly integrate into the MLonMCU flow. In addition a location in the users home directory (`~/.config/mlonmcu/models/` for Ubuntu) is also added to the search path as defined in the `environments.yml`. Feel free to add further model manually or alternatively clone a model-zoo (e.g. TODO:arm) to a random location on the disc and updating the list of model-paths in the environments' config file.

To list all available models in a environment, the `mlonmcu models` subcommand can be used:

In [ ]:
!mlonmcu models  # TODO: cleanup!

The listed "Paths" should match the search paths defined in the `environments.yml` and are processed in the provided order. If there are multiple models of the same name, only the first one will be added to MLonMCU. By passing the `--detailed` flag to the command, additional information will be shown for each entry.

TODO: allow to search/filter for models by passing an extra arg

In addition to individual models, predefined model-groups are also available. I.e. the MLPerf Tiny benchmark defined here:

In [ ]:
!cat $MLONMCU_HOME/models/groups.yml

Feel free to manualy define further model-groups for your needs.

### Running the MLonMCU flow

The MLonMCU flow is a major part of the tool and therefore explained by several examples, starting with minimal use cases before discussing more complex examples later.

**Invoke a single model on the default backend/target**

In [ ]:
!mlonmcu flow run resnet

There exist multiple stages which can be processed. The subcommand passed to `mlonmcu flow` defines the stage where the flow should stop. If you are only interested in intermediate artifacts of the process, use one of the following:

```
mlonmcu flow load  # only process the model with the frontend
mlonmcu flow build  # invoke the chosen backend to generate code
mlonmcu flow compile  # (cross-)compile the code for the chosen target
```

Further commands are available to achive further behavior:

```
mlonmcu flow test  # use provided model-data to invoke the models comparing their outputs with a golden reference
```

**Explicitly specify the backend and target to use for the flow**

In [ ]:
!mlonmcu flow run resnet --backend tvmaot --target host_x86

**Enable certain features for all defined runs**

In [ ]:
!mlonmcu flow run resnet --backend tvmaot --target etiss_pulpino --feature unpacked_api

**Add additional configuration options to customize internal components**

In [ ]:
!mlonmcu flow run resnet --backend tvmaot --config tvmaot.target_device=arm_cpu

Instead of defining certain key-value pass on aeach command line, feel free to use the `vars:` property defined in the `environment.yml` to achieve the same result.

**Invoke multiple models on multiple backends/targets**

In [ ]:
!mlonmcu flow run resnet toycar --backend tvmaot --backend tflmi --target host_x86 --target etiss_pulpino

**Exploit parallelism to reduce the processing time**

In [ ]:
!mlonmcu flow run resnet toycar --backend tvmaot --backend tflmi --target host_x86 --target etiss_pulpino --parallel 4

???

**Inspecting and exporting results and intermediate artifacts**

TODO

In [ ]:
mlonmcu sess or mlonmcu export -l?

In [ ]:
!mlonmcu export --session 0 --run 0 run.zip
!mlonmcu export --session 0 session.zip
!mlonmcu export session/

### Further MLonMCU usage

In [ ]:
see below?

## Demonstration (Python API)

### Preface

While it is also possible to initialize and setup an MLonMCU environment via Python, it is highly recommended to use the provided command line interfaces for these steps. The following examples demonstrate how to use the MLonMCU flow from within a Python program which enables more customizations of the internal features via scripting.

### Open a MLonMCU context to load the environment

### Running the MLonMCU flow

### Low level APIs

Core components of MLonMC (Frontends, Backends, Targets) can also be invoked individually as demonstrated in the following examples: